In [3]:
print("Jai sriram")
print("OM Namashivaya")

Jai sriram
OM Namashivaya


## Amazon Fine Food Review Analysis

Data Source :https://www.kaggle.com/snap/amazon-fine-food-reviews

Amazon fine food review consists of review of fine food from the Amazon

Number of reviews: 568,454
Number of users: 256,059
Number of products: 74,258
Timespan: Oct 1999 - Oct 2012
Number of Attributes/Columns in data: 10 

Attribute Information:
Id
ProductId - unique identifier for the product
UserId - unqiue identifier for the user
ProfileName
HelpfulnessNumerator - number of users who found the review helpful
HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
Score - rating between 1 and 5
Time - timestamp for the review
Summary - brief summary of the review
Text - text of the review

#### Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.



### Loading the data

The Dataset are available in two forms,

1) .csv file
2) SQLite Database

In order to load the data we have used SQLITE dataset as it is easier to query the data and visualize the data efficiently

Here as we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3. If the score id above 3, then the recommendation wil be set to "positive". Otherwise, it will be set to "negative".

In [4]:
#import libraries

import pandas as pd
import numpy as np
import nltk
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle


from tqdm import tqdm
import os

### Reading Data

In [11]:
con = sqlite3.connect("C:/Users/Falcon/Desktop/Keerthana/dataset/database.sqlite")
filtered_data = pd.read_sql_query(""" SELECT  * FROM reviews where score != 3 Limit 5000""",con)
filtered_data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [12]:
#Given a score <3 return negative review and score >3 return positive review

def partition(x):
    if x<3:
        return 0
    return 1

#apply the fuction to the column score in the dataframe
actualscore = filtered_data['Score']
pos_neg = actualscore.map(partition)
filtered_data['Score'] = pos_neg
print("Number of data points in our data",filtered_data.shape)
filtered_data.head(5)


Number of data points in our data (5000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [15]:
display = pd.read_sql_query("""SELECT UserId,ProductId,ProfileName,Time,Score,Text,Count(*) from Reviews GROUP by UserId 
having COUNT(*)>1""",con)

In [20]:
display2 = pd.read_sql_query("""SELECT UserId,ProductId,ProfileName,Time,Score,Text from Reviews where UserId ='#oc-R115TNMSPFT9I7'""",con)

In [26]:
display3 = pd.read_sql_query("""SELECT UserId,ProductId,ProfileName,Time,Score,Text from Reviews where UserId ='AZY10LLTJ71NX'""",con)

In [21]:
print(display2)

               UserId   ProductId ProfileName        Time  Score  \
0  #oc-R115TNMSPFT9I7  B005ZBZLT4     Breyton  1331510400      2   
1  #oc-R115TNMSPFT9I7  B007Y59HVM     Breyton  1331510400      2   

                                                Text  
0  Overall its just OK when considering the price...  
1  Overall its just OK when considering the price...  


In [22]:
display[display['UserId']=='AZY10LLTJ71NX']

,UserId,ProductId,ProfileName,Time,Score,Text,Count(*)
80638,AZY10LLTJ71NX,B001ATMQK2,"undertheshrine ""undertheshrine""",1296691200,5,I bought this 6 pack because for the price tha...,5


In [24]:
display2[display2['UserId']=='#oc-R115TNMSPFT9I7']

,UserId,ProductId,ProfileName,Time,Score,Text
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...
1,#oc-R115TNMSPFT9I7,B007Y59HVM,Breyton,1331510400,2,Overall its just OK when considering the price...


In [27]:
display3[display3['UserId']=='AZY10LLTJ71NX']

,UserId,ProductId,ProfileName,Time,Score,Text
0,AZY10LLTJ71NX,B001ATMQK2,"undertheshrine ""undertheshrine""",1296691200,5,I bought this 6 pack because for the price tha...
1,AZY10LLTJ71NX,B001P7AXXG,"undertheshrine ""undertheshrine""",1303776000,5,This popcorn is probably the best microwave po...
2,AZY10LLTJ71NX,B000MYW2ZA,"undertheshrine ""undertheshrine""",1334707200,5,I was recommended to try green tea extract to ...
3,AZY10LLTJ71NX,B0043CVIBG,"undertheshrine ""undertheshrine""",1303776000,5,if you love thin mint cookies that the girl sc...
4,AZY10LLTJ71NX,B006P7E5ZI,"undertheshrine ""undertheshrine""",1334707200,5,I was recommended to try green tea extract to ...


In [30]:
#number of customers who bought only one product in the entire data set

display11 = pd.read_sql_query("""SELECT UserId,ProductId,ProfileName,Time,Score,Text,Count(*) from Reviews GROUP by UserId 
having COUNT(*)=1""",con)

In [29]:
display11.shape

(175391, 7)

# Exploratory Data Analysis

## [2] Data Cleaning: Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.  Following is an example:

In [37]:
display = pd.read_sql_query("""
select * from Reviews where Score !=3 and UserId = "AR5J8UI46CURR" order by ProductId """,con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As can be seen above the same user has multiple reviews of the with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text  and on doing analysis it was found that <br>
<br> 
ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [39]:
#Link to dataframe.sort_values :https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.sort_values.html


#Sorting data according to the productId in ascendind values
sorted_data = filtered_data.sort_values('ProductId',axis=0,ascending=True,inplace=False,kind='quicksort',na_position='last')

In [41]:
#link to dataframe.drop_duplicates  :https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html

#Deduplication of entries
final = sorted_data.drop_duplicates(subset = {'UserId','ProfileName','Time','Text'},keep="first",inplace=False)
final.shape

(4986, 10)

In [ ]:
#checking how much percentage of data still remains

